# Dynamic Graph CNN

In [1]:
!pip install torch torchvision torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric fastai

## Data loading
Let's get the dataset

In [0]:
import torch
from torch_geometric.datasets import ModelNet
import torch_geometric.transforms as T

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

pre_transform = T.NormalizeScale()

train_ds = ModelNet(root='/',
             train=True,
             pre_transform=pre_transform)

test_ds = ModelNet(root='/',
             train=True,
             pre_transform=pre_transform)

Now we have to define our dataloader, these guys will handle the thread queue to feed the GPU

In [0]:
from torch_geometric.data import DataLoader

train_dl = DataLoader(train_ds, batch_size=3)

test_dl = DataLoader(test_ds, batch_size=2)

## Model

Define our architecture

In [0]:
import torch
import torch.nn as nn
import torch_geometric.nn as gnn
from torch.nn import Sequential, Linear, ReLU
from torch_geometric.nn import EdgeConv, knn_graph, global_max_pool
from torch_geometric.nn import knn_graph
from torch.nn import Sequential as Seq, Linear as Lin, ReLU
import torch.nn.functional as F

class DynamicEdgeConv(gnn.EdgeConv):
    def __init__(self, k=6, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.k = k

    def forward(self, pos, batch):
        edge_index = knn_graph(pos, self.k, batch, loop=False)
        return super().forward(pos, edge_index)

class DGCNNClassification(nn.Module):
  def __init__(self, in_channels, n_classes, k=20):
    super(DGCNNClassification, self).__init__()

    self.convs = nn.ModuleList([
        DynamicEdgeConv(
            k=k,
            nn=Sequential(
              Linear(in_channels * 2, 64),
              ReLU(),
              Linear(64, 64),
              ReLU(),
              Linear(64, 64),
              ReLU()
            ), 
            aggr='max'),
        DynamicEdgeConv(
            k=k,
            nn=Sequential(
              Linear(64 * 2, 128),
              ReLU()
          ), 
        aggr='max') 
    ])
    
    
    self.point_wise_features2higher_dim = nn.Sequential(
        nn.Linear(128, 512),
        nn.ReLU()
    )
    
    self.tail = nn.Sequential(
#         nn.Linear(1024, 512),
        nn.Linear(512, 256),
        nn.ReLU(),
        nn.Linear(256, n_classes),

    )
    
    self.k = k
    
  def forward(self, x, batch):
      out = x
      for conv in self.convs:
        out = conv(out, batch)
      
      out = self.point_wise_features2higher_dim(out)
      out = global_max_pool(out, batch)
      out = self.tail(out)
      
      return out

In [7]:
for batch in train_dl:
  print(batch)
  break

Batch(batch=[3204], face=[3, 4055], pos=[3204, 3], y=[3])


## Training

In [0]:
from torch.optim import Adam
import time


model = DGCNNClassification(3,10).to(device)


optimizer = Adam(model.parameters(), 0.001)
criterion = nn.CrossEntropyLoss()

for data in train_dl:
    start = time.time()
    optimizer.zero_grad()
    data = data.to(device)
    print(data)
    out = model(data.pos, data.batch)
    loss = criterion(out, data.y)
    loss.backward()
    print('[INFO] loss={} elapsed={:.2f}'.format(loss.item(), time.time() - start))
    optimizer.step()


Batch(batch=[36454], face=[3, 58136], pos=[36454, 3], y=[3])
[INFO] loss=2.287642240524292 elapsed=245.00
Batch(batch=[26877], face=[3, 18136], pos=[26877, 3], y=[3])
[INFO] loss=2.2878687381744385 elapsed=77.72
Batch(batch=[4754], face=[3, 5411], pos=[4754, 3], y=[3])
[INFO] loss=2.222356081008911 elapsed=2.94
Batch(batch=[3601], face=[3, 3554], pos=[3601, 3], y=[3])
[INFO] loss=2.230229139328003 elapsed=1.63
Batch(batch=[4024], face=[3, 3482], pos=[4024, 3], y=[3])
[INFO] loss=2.337280511856079 elapsed=2.67
Batch(batch=[16465], face=[3, 12979], pos=[16465, 3], y=[3])
[INFO] loss=2.2916717529296875 elapsed=14.24
Batch(batch=[40478], face=[3, 27840], pos=[40478, 3], y=[3])
[INFO] loss=2.3007266521453857 elapsed=367.46
Batch(batch=[12947], face=[3, 13608], pos=[12947, 3], y=[3])
[INFO] loss=2.3099775314331055 elapsed=10.75
Batch(batch=[125945], face=[3, 73000], pos=[125945, 3], y=[3])
[INFO] loss=2.15262770652771 elapsed=2562.24
Batch(batch=[990], face=[3, 734], pos=[990, 3], y=[3])
[IN

## Traversability

In [0]:

x = np.ones((76, 76)) # this should be an image from somewhere
x = torch.tensor(x, dtype=torch.float).unsqueeze(-1)

edge_index, pos = grid(76,76)

data = Data(x=x, edge_index=edge_index, pos=pos.to(torch.float))
data